<a href="https://colab.research.google.com/github/Alao001/Medical-Artices-Summarizer/blob/main/Medical_Articles_Summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!nvidia-smi

Sat Oct 19 12:47:18 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P8              13W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

Installing Dependencies

In [4]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 89.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 86.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.7/413.7 kB 35.3 MB/s eta 0:00:00
   ━━━━━

 Importing Libraries and Setting Up the Environment

In [5]:
from transformers import pipeline, set_seed
from datasets import load_dataset, load_from_disk
import matplotlib.pyplot as plt
from datasets import load_dataset
import pandas as pd
import evaluate
from datasets import load_dataset

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import nltk
from nltk.tokenize import sent_tokenize
from tqdm import tqdm
import torch

# Download NLTK data
nltk.download("punkt")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
# Check if CUDA is available, use GPU if available, otherwise use CPU
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

Model Description

The T5 Large for Medical Text Summarization is a specialized variant of the T5 transformer model, fine-tuned for the task of summarizing medical text. This model is designed to generate concise and coherent summaries of medical documents, research papers, clinical notes, and other healthcare-related text.

In [7]:
# Load the medical model and tokenizer
model = "Falconsai/medical_summarization"
tokenizer = AutoTokenizer.from_pretrained(model)
model_medical = AutoModelForSeq2SeqLM.from_pretrained(model).to(device)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.37k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [8]:
# Load the dataset
dataset_pubmed = load_dataset("ccdv/pubmed-summarization")

README.md:   0%|          | 0.00/3.80k [00:00<?, ?B/s]

train-00000-of-00005.parquet:   0%|          | 0.00/210M [00:00<?, ?B/s]

train-00001-of-00005.parquet:   0%|          | 0.00/208M [00:00<?, ?B/s]

train-00002-of-00005.parquet:   0%|          | 0.00/207M [00:00<?, ?B/s]

train-00003-of-00005.parquet:   0%|          | 0.00/211M [00:00<?, ?B/s]

...

validation-00000-of-00001.parquet:   0%|          | 0.00/59.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/58.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/119924 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/6633 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6658 [00:00<?, ? examples/s]

In [9]:
dataset_pubmed

DatasetDict({
    train: Dataset({
        features: ['article', 'abstract'],
        num_rows: 119924
    })
    validation: Dataset({
        features: ['article', 'abstract'],
        num_rows: 6633
    })
    test: Dataset({
        features: ['article', 'abstract'],
        num_rows: 6658
    })
})

In [10]:
# Print dataset information
split_lengths = [len(dataset_pubmed[split]) for split in dataset_pubmed]
print(f"Dataset Features: {dataset_pubmed['train'].column_names}")


Dataset Features: ['article', 'abstract']


In [1]:
print("\nSample Article:")
print(dataset_pubmed["train"][1]["article"])
print("\nSample Abstract:")
print(dataset_pubmed["train"][1]["abstract"])


Sample Article:


NameError: name 'dataset_pubmed' is not defined

In [12]:
# Define a function to convert examples to features
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch['article'], max_length=1024, truncation=True)

    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['abstract'], max_length=128, truncation=True)

    return {
        'input_ids': input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }

In [13]:
# Map the dataset to features
dataset_pubmed_pt = dataset_pubmed.map(convert_examples_to_features, batched=True)

Map:   0%|          | 0/119924 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/6633 [00:00<?, ? examples/s]

Map:   0%|          | 0/6658 [00:00<?, ? examples/s]

In [14]:
dataset_pubmed_pt['train']

Dataset({
    features: ['article', 'abstract', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 119924
})

In [15]:
dataset_pubmed_pt['train']['input_ids'][1]

[34,
 6986,
 16,
 72,
 145,
 5743,
 13,
 1221,
 11,
 164,
 1535,
 12669,
 16,
 824,
 1308,
 13,
 1874,
 7,
 3,
 6,
 902,
 16,
 1221,
 3,
 22725,
 26324,
 11,
 87,
 127,
 11423,
 3918,
 5,
 536,
 46,
 11658,
 19,
 4802,
 38,
 46,
 22666,
 3,
 30715,
 593,
 13,
 24731,
 14063,
 77,
 41,
 3,
 107,
 115,
 3,
 61,
 41,
 3,
 107,
 115,
 3,
 2,
 586,
 3,
 122,
 3,
 87,
 3,
 26,
 40,
 3,
 61,
 11,
 164,
 7931,
 38,
 3,
 9,
 741,
 13,
 8,
 3,
 10067,
 1994,
 3,
 6,
 19021,
 3,
 6,
 2714,
 7470,
 3,
 6,
 26324,
 3,
 6,
 42,
 11423,
 3918,
 3,
 5,
 17413,
 2116,
 3130,
 24,
 9990,
 11,
 2072,
 32,
 3,
 18,
 3518,
 610,
 227,
 11423,
 3918,
 3,
 6,
 902,
 16,
 819,
 11,
 5378,
 1874,
 7,
 3,
 6,
 164,
 36,
 22001,
 57,
 46,
 11658,
 5,
 2266,
 46,
 11658,
 557,
 4131,
 29,
 7,
 3976,
 224,
 38,
 13034,
 3,
 6,
 18724,
 3,
 6,
 11,
 16633,
 102,
 29,
 15,
 9,
 3,
 6,
 11,
 2932,
 164,
 43,
 3,
 9,
 2841,
 1504,
 30,
 463,
 13,
 280,
 41,
 3,
 1824,
 32,
 40,
 3,
 61,
 11,
 821,
 2637,
 16,
 1221,
 

In [16]:
dataset_pubmed_pt['train']['attention_mask'][1]

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [17]:
dataset_pubmed_pt['train']['labels'][1]

[2458,
 152,
 11658,
 16,
 1221,
 28,
 1874,
 113,
 33,
 3,
 22725,
 1676,
 3918,
 19,
 5871,
 15110,
 11,
 164,
 4131,
 29,
 463,
 13,
 280,
 16,
 175,
 1221,
 3,
 5,
 8,
 1504,
 13,
 1717,
 3017,
 7316,
 19,
 557,
 7234,
 11,
 164,
 36,
 1968,
 28,
 2261,
 11233,
 984,
 3,
 5,
 3,
 4203,
 8514,
 102,
 32,
 725,
 159,
 3,
 18,
 24664,
 4373,
 33,
 59,
 1231,
 16,
 10738,
 1752,
 1454,
 13,
 1221,
 11,
 164,
 43,
 3,
 9,
 2841,
 1504,
 30,
 1879,
 9990,
 5,
 8345,
 235,
 6570,
 8,
 13577,
 447,
 4710,
 11,
 28297,
 13,
 6344,
 162,
 10529,
 3575,
 3918,
 16,
 1221,
 28,
 1874,
 113,
 43,
 529,
 3,
 18,
 3575,
 3,
 18,
 20,
 4638,
 23,
 4392,
 46,
 11658,
 11,
 113,
 33,
 3,
 22725,
 1058,
 28,
 26324,
 406,
 8,
 169,
 13,
 3,
 4203,
 8514,
 102,
 1]

Preparing batches of data in a format suitable for training the model

In [18]:
from transformers import DataCollatorForSeq2Seq

# Initialize data collator for seq2seq
seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_medical)

In [19]:
# Define training arguments

from transformers import TrainingArguments, Trainer
trainer_args = TrainingArguments(
    output_dir='medical-article-summarizer', num_train_epochs=1, warmup_steps=500,
    per_device_train_batch_size=1, per_device_eval_batch_size=1,
    weight_decay=0.01, logging_steps=10,
    evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
    gradient_accumulation_steps=16
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [20]:
# Initialize Trainer
trainer = Trainer(model=model_medical, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_pubmed_pt["test"],
                  eval_dataset=dataset_pubmed_pt["validation"])

In [21]:
# Train the model
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss


TrainOutput(global_step=416, training_loss=2.141383171081543, metrics={'train_runtime': 966.9257, 'train_samples_per_second': 6.886, 'train_steps_per_second': 0.43, 'total_flos': 1784365343047680.0, 'train_loss': 2.141383171081543, 'epoch': 0.99969960949234})

Evaluation

In [31]:
def generate_batch_sized_chunks(list_of_elements, batch_size):
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i: i + batch_size]

# Define a function to calculate ROUGE metric on the test dataset
def calculate_metric_on_test_ds(dataset, rouge, model, tokenizer,
                                batch_size=16, device=device,
                                column_text="article", column_summary="highlights"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
            zip(article_batches, target_batches), total=len(article_batches)):
        inputs = tokenizer(article_batch, max_length=1024, truncation=True,
                           padding="max_length", return_tensors="pt")

        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                                   attention_mask=inputs["attention_mask"].to(device),
                                   length_penalty=0.8, num_beams=8, max_length=128)

        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                              clean_up_tokenization_spaces=True)
                             for s in summaries]

        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]

        rouge.add_batch(predictions=decoded_summaries, references=target_batch)

    score = rouge.compute()
    return score

In [39]:
import evaluate

rouge_metric = evaluate.load('rouge')

# Calculate ROUGE scores on the test dataset
score = calculate_metric_on_test_ds(dataset_pubmed['test'][0:10], rouge_metric, model_medical, tokenizer,
                                    column_text='article', column_summary='abstract', batch_size=8)

rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
print(score["rouge1"])
print(score["rouge2"])
print(score["rougeL"])
print(score["rougeLsum"])
#rouge_dict = dict((rn, score[rn]['fmeasure']) for rn in rouge_names)
#rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
#pd.DataFrame(rouge_names, index=[f'pubmed'])

100%|██████████| 2/2 [00:11<00:00,  5.72s/it]


0.014565784220427334
0.0
0.01350986058741549
0.014320263243525384


In [40]:
# save model
model_medical.save_pretrained("medical_summary_model")

In [41]:
# save tokenizer
tokenizer.save_pretrained("tokenizer")

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/spiece.model',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [42]:
# load
tokenizer = AutoTokenizer.from_pretrained("/content/tokenizer")

In [43]:
# prediction
gen_kwargs = {"length_penalty": 0.8, "num_beams": 8, "max_length": 128}

sample_text = dataset_pubmed["test"][0]["article"]
reference = dataset_pubmed["test"][0]["abstract"]

pipe = pipeline("summarization", model="medical_summary_model", tokenizer=tokenizer)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [45]:
print("Article:")
print(sample_text)

print("\nReference Abstract:")
print(reference)

print("\nModel Abstract:")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])

Article:
anxiety affects quality of life in those living with parkinson 's disease ( pd ) more so than overall cognitive status , motor deficits , apathy , and depression [ 13 ] . 
 although anxiety and depression are often related and coexist in pd patients , recent research suggests that anxiety rather than depression is the most prominent and prevalent mood disorder in pd [ 5 , 6 ] . yet , 
 our current understanding of anxiety and its impact on cognition in pd , as well as its neural basis and best treatment practices , remains meager and lags far behind that of depression . 
 overall , neuropsychiatric symptoms in pd have been shown to be negatively associated with cognitive performance . 
 for example , higher depression scores have been correlated with lower scores on the mini - mental state exam ( mmse ) [ 8 , 9 ] as well as tests of memory and executive functions ( e.g. , attention ) [ 1014 ] . 
 likewise , apathy and anhedonia in pd patients have been associated with executiv